In [1]:
import pandas as pd 
import numpy as np
from scipy import stats
import scipy.optimize
from scipy.optimize import OptimizeWarning
import warnings
import math
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import mplfinance
from matplotlib.dates import date2num
from datetime import datetime




In [4]:
class holder:
    1

In [3]:
#class holder:
    
   #Heiken Ashi Function
def heikenashi(prices,periods):
       
    
        #results = holder()
        
        #dict={}
        
        HAclose = prices[['open','high','low','close']].sum(axis=1)/4
        HAopen = HAclose.copy()
        HAopen.iloc[0]=HAclose.iloc[0]
        HAhigh = HAclose.copy()
        HAlow= HAclose.copy()
        
        for i in range(1,len(prices)):
            
            HAopen.iloc[i] = (HAopen.iloc[i-1]+HAclose.iloc[i-1])/2
            HAhigh.iloc[i] = np.array((prices.high.iloc[i],HAopen.iloc[i],HAclose.iloc[i])).max()
            HAlow.iloc[i] = np.array((prices.low.iloc[i],HAopen.iloc[i],HAclose.iloc[i])).min()
    
        df = pd.concat((HAopen,HAhigh,HAlow,HAclose),axis=1)
        df.columns=['HAopen','HAhigh','HAlow','HAclose']
        #df.columns=[['Open','High','Low','Close']]
        
        #df.index = df.index.droplevel(0)
        #dict[periods[0]] = df
        #results.candles = dict
        
        return df


In [4]:
def detrend(prices):
    
    detrended = prices.close[1:]-prices.close[:-1].values
    
    return detrended

In [5]:
#Fseries
def fseries(x,a0,a1,b1,w):
    f = a0 + a1*np.cos(w*x)+b1*np.sin(w*x)
    return f

In [6]:
#serie de fourier

def fourier(prices,periods):
    
    plot = True
    detrended = detrend(prices)
    
    for i in range(0,len(periods)):
        coeffs=[]
        
        for j in range(periods[i],len(prices)-periods[i]):
            x=np.arange(0,periods[i])
            y=detrended.iloc[j-periods[i]:j]
            
            with warnings.catch_warnings():
                warnings.simplefilter('error',OptimizeWarning)
                
                try:
                    res = scipy.optimize.curve_fit(fseries,x,y)
                    
                except (RuntimeError,OptimizeWarning):
                    res=np.empty((1,4))
                    res[0,:]=np.NAN
                    
            if plot == True :
                xt = np.linspace(0,periods[i],100)
                yt = fseries(xt,res[0][0],res[0][1],res[0][2],res[0][3])
                
                plt.plot(x,y)
                plt.plot(xt,yt,'r')
                plt.show()
                
            coeffs=np.append(coeffs,res[0],axis=0)
        
        warnings.filterwarnings('ignore',category=np.VisibleDeprecationWarning)
        
        coeffs = np.array(coeffs).reshape(((len(coeffs)/4,4)))
        
        df = pd.DataFrame(coeffs,index=prices.iloc[periods[i]:-periods[i]])
        
        df.columns=[['a0','a1','b1','w']]
        
        df=df.fillna(method='bfill')
        
    return df

In [2]:
#WADL
def wadl(prices,periods):
    
    results=holder()
    dict={}
    for i in range(0,len(periods)):
        WAD=[]
        for j in range(periods[i],len(prices)-periods[i]):
            TRH = np.array([prices.high.iloc[j]-prices.close.iloc[j-1]]).max()
            TRL = np.array([prices.low.iloc[j]-prices.close.iloc[j-1]]).min()
        
            if prices.close.iloc[j] > prices.close.iloc[j-1]:
                PM = prices.close.iloc[j] - TRL
            
            elif prices.close.iloc[j] < prices.close.iloc[j-1]:
                    PM = prices.close.iloc[j] - TRH
        
            elif prices.close.iloc[j] == prices.close.iloc[j-1]:
                PM=0
            else :
                print('error !!')
            
            AD = PM * prices.volume.iloc[j]
        
            WAD = np.append(WAD,AD)
        
        WAD = WAD.cumsum()
        
        WAD = pd.DataFrame(WAD,index=prices.iloc[periods[i]:-periods[i]].index)
        
        WAD.columns=['close']
        
        dict[periods[i]] = WAD
        
    results.wadl = dict
        
    return results
        
        
        
        

In [5]:
#Momentum function 
def momentum(prices,periods):
    
    results= holder()
    open={}
    close={}
    
    for i in range(0,len(periods)):
        open[periods[i]] = pd.DataFrame(prices.open.iloc[periods[i]:] - prices.open.iloc[:-periods[i]].values,
                                       index= prices.open.iloc[periods[i]:].index )
        close[periods[i]] = pd.DataFrame(prices.close.iloc[periods[i]:] - prices.close.iloc[:-periods[i]].values,
                                       index= prices.close.iloc[periods[i]:].index )
        open[periods[i]].columns = ['open']
        close[periods[i]].columns = ['close']
    
    results.open = open
    results.close = close
    
    return results